In [2]:
from pyspark import SparkConf, SparkContext
import os
#os.environ['PYSPARK_PYTHON'] = '/Library/Frameworks/Python.framework/Versions/3.6/bin/python3'
os.environ['PYSPARK_PYTHON'] = '/usr/local/Cellar/python@3.10/3.10.9/bin/python3'

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("sparksql").getOrCreate()

#sc.stop()
conf = SparkConf().setMaster("local").setAppName("sparksql")
sc = SparkContext.getOrCreate()

24/12/23 21:39:07 WARN Utils: Your hostname, SANDEEPs-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.1.105 instead (on interface en0)
24/12/23 21:39:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/23 21:39:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/12/23 21:39:08 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/12/23 21:39:08 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/12/23 21:39:08 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [3]:
#sc
#spark.stop()

## Spark can internally convert any object into a dataframe and treat it as if it were a table, and hence run SQLs on it. All functions remain available

In [4]:
flightData2015 = spark\
.read\
.option("inferSchema", "true")\
.option("header", "true")\
.csv("/Users/aakash/Downloads/spark-code/data/flight-data/csv/2015-summary.csv")

AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/Users/aakash/Downloads/spark-code/data/flight-data/csv/2015-summary.csv.

In [3]:
flightData2015.take(1)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15)]

In [4]:
## Register the dataframe as a table
flightData2015.createOrReplaceTempView("flight_data_2015")

In [6]:
#type(flight_data_2015)

In [5]:
sqlWay = spark.sql("""
SELECT DEST_COUNTRY_NAME, count(*)
FROM flight_data_2015
GROUP BY DEST_COUNTRY_NAME
""")


dataFrameWay = flightData2015\
.groupBy("DEST_COUNTRY_NAME")\
.count()

sqlWay.explain()
dataFrameWay.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[DEST_COUNTRY_NAME#17], functions=[count(1)])
   +- Exchange hashpartitioning(DEST_COUNTRY_NAME#17, 200), ENSURE_REQUIREMENTS, [plan_id=39]
      +- HashAggregate(keys=[DEST_COUNTRY_NAME#17], functions=[partial_count(1)])
         +- FileScan csv [DEST_COUNTRY_NAME#17] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/Users/aakash/Downloads/spark-code/data/flight-data/csv/2015-summ..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>


== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[DEST_COUNTRY_NAME#17], functions=[count(1)])
   +- Exchange hashpartitioning(DEST_COUNTRY_NAME#17, 200), ENSURE_REQUIREMENTS, [plan_id=52]
      +- HashAggregate(keys=[DEST_COUNTRY_NAME#17], functions=[partial_count(1)])
         +- FileScan csv [DEST_COUNTRY_NAME#17] Batched: false, DataFilters: [], Format: CSV, Location: In

In [6]:
sqlWay.count()

132

In [7]:
dataFrameWay = flightData2015\
.groupBy("DEST_COUNTRY_NAME")\
.count()

dataFrameWay.take(1)

[Row(DEST_COUNTRY_NAME='Anguilla', count=1)]

In [8]:
from pyspark.sql.functions import desc
flightData2015\
.groupBy("DEST_COUNTRY_NAME")\
.sum("count")\
.withColumnRenamed("sum(count)", "destination_total")\
.sort(desc("destination_total"))\
.limit(5)\
.take(5)

[Row(DEST_COUNTRY_NAME='United States', destination_total=411352),
 Row(DEST_COUNTRY_NAME='Canada', destination_total=8399),
 Row(DEST_COUNTRY_NAME='Mexico', destination_total=7140),
 Row(DEST_COUNTRY_NAME='United Kingdom', destination_total=2025),
 Row(DEST_COUNTRY_NAME='Japan', destination_total=1548)]

In [9]:
sqlWay = spark.sql("""
CREATE TABLE flights_2_new (
DEST_COUNTRY_NAME STRING, 
ORIGIN_COUNTRY_NAME STRING, 
count LONG)
USING JSON OPTIONS (path '/Users/aakash/Downloads/spark-code/data/flight-data/json/2015-summary.json')
"""
)

In [10]:
sqlWay.createGlobalTempView('flights_2_new')

In [11]:
sqlWay=spark.sql("""
SELECT * FROM flights_2_new
""")
sqlWay.show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Romania|   15|
|       United States|            Croatia|    1|
|       United States|            Ireland|  344|
|               Egypt|      United States|   15|
|       United States|              India|   62|
|       United States|          Singapore|    1|
|       United States|            Grenada|   62|
|          Costa Rica|      United States|  588|
|             Senegal|      United States|   40|
|             Moldova|      United States|    1|
|       United States|       Sint Maarten|  325|
|       United States|   Marshall Islands|   39|
|              Guyana|      United States|   64|
|               Malta|      United States|    1|
|            Anguilla|      United States|   41|
|             Bolivia|      United States|   30|
|       United States|           Paraguay|    6|
|             Algeri

In [12]:
### BETWEEN
countsql=spark.sql("SELECT * FROM flights_2_new WHERE count BETWEEN 40 AND 100")
countsql.show()

+--------------------+--------------------+-----+
|   DEST_COUNTRY_NAME| ORIGIN_COUNTRY_NAME|count|
+--------------------+--------------------+-----+
|       United States|               India|   62|
|       United States|             Grenada|   62|
|             Senegal|       United States|   40|
|              Guyana|       United States|   64|
|            Anguilla|       United States|   41|
|       United States|Federated States ...|   69|
|    Marshall Islands|       United States|   42|
|       United States|             Senegal|   42|
|          Martinique|       United States|   44|
|       United States|    Saint Barthelemy|   41|
|Federated States ...|       United States|   69|
|       United States|          Guadeloupe|   59|
|               India|       United States|   61|
|       United States|         New Zealand|   74|
|    French Polynesia|       United States|   43|
|       United States|          Martinique|   43|
|       United States|             Nigeria|   50|


# store a dataframe as table

In [14]:
dataFrameWay.write.bucketBy(42, "DEST_COUNTRY_NAME").sortBy("count").saveAsTable("flight_in_table_22")

In [15]:
tabsqlWay = spark.sql("""
SELECT *
FROM flight_in_table_22
""")
tabsqlWay.show()

+--------------------+-----+
|   DEST_COUNTRY_NAME|count|
+--------------------+-----+
|Saint Vincent and...|    1|
|Turks and Caicos ...|    1|
|            Dominica|    1|
|               Ghana|    1|
|          Martinique|    1|
|Federated States ...|    1|
|            Bulgaria|    1|
|            Paraguay|    1|
|              France|    1|
|             Algeria|    1|
|    Saint Barthelemy|    1|
|            Ethiopia|    1|
|        Burkina Faso|    1|
|             Bahrain|    1|
|              Taiwan|    1|
|       Cote d'Ivoire|    1|
| Trinidad and Tobago|    1|
|              Poland|    1|
|            Pakistan|    1|
|               Italy|    1|
+--------------------+-----+
only showing top 20 rows



In [16]:
# read data directly from a parquet file
spark.sql("select * from parquet.`/Users/aakash/Downloads/spark-code/data/flight-data/parquet/2010-summary.parquet`").show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Romania|    1|
|       United States|            Ireland|  264|
|       United States|              India|   69|
|               Egypt|      United States|   24|
|   Equatorial Guinea|      United States|    1|
|       United States|          Singapore|   25|
|       United States|            Grenada|   54|
|          Costa Rica|      United States|  477|
|             Senegal|      United States|   29|
|       United States|   Marshall Islands|   44|
|              Guyana|      United States|   17|
|       United States|       Sint Maarten|   53|
|               Malta|      United States|    1|
|             Bolivia|      United States|   46|
|            Anguilla|      United States|   21|
|Turks and Caicos ...|      United States|  136|
|       United States|        Afghanistan|    2|
|Saint Vincent and..

In [17]:
spark.sql("SELECT date_add('2016-07-30', 1)").show()

+-----------------------+
|date_add(2016-07-30, 1)|
+-----------------------+
|             2016-07-31|
+-----------------------+



In [18]:
spark.sql("SELECT date_format('2016-04-08', 'y')").show()

+--------------------------+
|date_format(2016-04-08, y)|
+--------------------------+
|                      2016|
+--------------------------+



In [19]:
spark.sql("SELECT date_part('YEAR', TIMESTAMP '2019-08-12 01:00:00.123456')").show()

+-------------------------------------------------------+
|date_part(YEAR, TIMESTAMP '2019-08-12 01:00:00.123456')|
+-------------------------------------------------------+
|                                                   2019|
+-------------------------------------------------------+



In [20]:
spark.sql("SELECT element_at(array(1, 2, 3), 2)").show()

+-----------------------------+
|element_at(array(1, 2, 3), 2)|
+-----------------------------+
|                            2|
+-----------------------------+



In [21]:
spark.sql("SELECT endswith('Spark SQL', 'SQL')").show()

+------------------------+
|endswith(Spark SQL, SQL)|
+------------------------+
|                    true|
+------------------------+



In [22]:
# returns true is every expression is true
spark.sql("SELECT every(col) FROM VALUES (true), (true), (true) AS tab(col)").show()

+----------+
|every(col)|
+----------+
|      true|
+----------+



In [30]:
spark.sql("SELECT explode(array(10, 20))").show()

+---+
|col|
+---+
| 10|
| 20|
+---+



In [31]:
spark.sql("SELECT find_in_set('ab','abc,b,ab,c,def');").show()

+-------------------------------+
|find_in_set(ab, abc,b,ab,c,def)|
+-------------------------------+
|                              3|
+-------------------------------+



In [35]:
spark.sql("SELECT from_json('{\"a\":1, \"b\":0.8}', 'a INT, b DOUBLE')").show()

+---------------------------+
|from_json({"a":1, "b":0.8})|
+---------------------------+
|                   {1, 0.8}|
+---------------------------+



In [37]:
spark.sql("SELECT isnan(cast('NaN' as double))").show()

+--------------------------+
|isnan(CAST(NaN AS DOUBLE))|
+--------------------------+
|                      true|
+--------------------------+



In [38]:
spark.sql("SELECT isnotnull(1)").show()

+---------------+
|(1 IS NOT NULL)|
+---------------+
|           true|
+---------------+



In [39]:
spark.sql("SELECT last_value(col) FROM VALUES (10), (5), (20) AS tab(col)").show()

+---------------+
|last_value(col)|
+---------------+
|             20|
+---------------+



In [40]:
spark.sql("SELECT levenshtein('kitten', 'sitting')").show()

+----------------------------+
|levenshtein(kitten, sitting)|
+----------------------------+
|                           3|
+----------------------------+



In [77]:
spark.sql("SELECT lpad('hi', 5, '?')").show()

+--------------+
|lpad(hi, 5, ?)|
+--------------+
|         ???hi|
+--------------+



In [42]:
spark.sql("SELECT make_date(2013, 7, 15)").show()

+----------------------+
|make_date(2013, 7, 15)|
+----------------------+
|            2013-07-15|
+----------------------+



In [43]:
spark.sql("SELECT map(1.0, '2', 3.0, '4')").show()

+--------------------+
| map(1.0, 2, 3.0, 4)|
+--------------------+
|{1.0 -> 2, 3.0 -> 4}|
+--------------------+



In [78]:
spark.sql("SELECT map_contains_key(map(1, 'a', 2, 'b'), 5)").show()

+------------------------------------+
|map_contains_key(map(1, a, 2, b), 5)|
+------------------------------------+
|                               false|
+------------------------------------+



In [45]:
spark.sql("SELECT map_filter(map(1, 0, 2, 2, 3, -1), (k, v) -> k > v)").show()

+-------------------------------------------------------------------------------------------------------------------------------------------------+
|map_filter(map(1, 0, 2, 2, 3, -1), lambdafunction((namedlambdavariable() > namedlambdavariable()), namedlambdavariable(), namedlambdavariable()))|
+-------------------------------------------------------------------------------------------------------------------------------------------------+
|                                                                                                                                {1 -> 0, 3 -> -1}|
+-------------------------------------------------------------------------------------------------------------------------------------------------+



In [46]:
spark.sql("SELECT map_keys(map(1, 'a', 2, 'b'))").show()

+-------------------------+
|map_keys(map(1, a, 2, b))|
+-------------------------+
|                   [1, 2]|
+-------------------------+



In [47]:
spark.sql("SELECT map_values(map(1, 'a', 2, 'b'))").show()

+---------------------------+
|map_values(map(1, a, 2, b))|
+---------------------------+
|                     [a, b]|
+---------------------------+



In [49]:
# retunrs null if expr1=expr2
spark.sql("SELECT nullif(2, 3)").show()

+------------+
|nullif(2, 3)|
+------------+
|           2|
+------------+



In [50]:
spark.sql("SELECT octet_length('Spark SQL')").show()

+-----------------------+
|octet_length(Spark SQL)|
+-----------------------+
|                      9|
+-----------------------+



In [51]:
spark.sql("SELECT percentile(col, 0.3) FROM VALUES (0), (10) AS tab(col)").show()

+-----------------------+
|percentile(col, 0.3, 1)|
+-----------------------+
|                    3.0|
+-----------------------+



In [15]:
#returns positive value of expr1 mod expr2
spark.sql("SELECT pmod(-10, 3)").show()

+------------+
|pmod(-10, 3)|
+------------+
|           2|
+------------+



In [53]:
spark.sql("SELECT random()").show()

+-------------------+
|             rand()|
+-------------------+
|0.04980729611873558|
+-------------------+



In [3]:
spark.sql("SELECT a, b, rank(b) OVER (PARTITION BY a ORDER BY b) as rnk FROM VALUES ('A1', 2),  \
          ('A1', 1), ('A2', 3), ('A1', 1) tab(a, b)").show()

+---+---+---+
|  a|  b|rnk|
+---+---+---+
| A1|  1|  1|
| A1|  1|  1|
| A1|  2|  3|
| A2|  3|  1|
+---+---+---+



In [58]:
spark.sql("SELECT schema_of_json('[{\"col\":0}]')").show()

+---------------------------+
|schema_of_json([{"col":0}])|
+---------------------------+
|       ARRAY<STRUCT<col:...|
+---------------------------+



In [59]:
spark.sql("SELECT sha1('Spark')").show()

+--------------------+
|         sha1(Spark)|
+--------------------+
|85f5955f4b27a9a4c...|
+--------------------+



In [60]:
spark.sql("SELECT shiftleft(2, 1)").show()

+---------------+
|shiftleft(2, 1)|
+---------------+
|              4|
+---------------+



In [61]:
spark.sql("SELECT shuffle(array(1, 20, 3, 5))").show()

+---------------------------+
|shuffle(array(1, 20, 3, 5))|
+---------------------------+
|              [1, 20, 3, 5]|
+---------------------------+



In [62]:
spark.sql("SELECT skewness(col) FROM VALUES (-10), (-20), (100), (1000) AS tab(col)").show()

+------------------+
|     skewness(col)|
+------------------+
|1.1135657469022011|
+------------------+



In [63]:
spark.sql("SELECT sort_array(array('b', 'd', null, 'c', 'a'), true)").show()

+-----------------------------------------+
|sort_array(array(b, d, NULL, c, a), true)|
+-----------------------------------------+
|                       [null, a, b, c, d]|
+-----------------------------------------+



In [67]:
spark.sql("SELECT translate('AaBbCc', 'abc', '123')").show()

+---------------------------+
|translate(AaBbCc, abc, 123)|
+---------------------------+
|                     A1B2C3|
+---------------------------+



In [68]:
spark.sql("SELECT typeof(1)").show()

+---------+
|typeof(1)|
+---------+
|      int|
+---------+



In [69]:
spark.sql("SELECT xpath('<a><b>b1</b><b>b2</b><b>b3</b><c>c1</c><c>c2</c></a>','a/b/text()')").show()

+-----------------------------------------------------------------------+
|xpath(<a><b>b1</b><b>b2</b><b>b3</b><c>c1</c><c>c2</c></a>, a/b/text())|
+-----------------------------------------------------------------------+
|                                                           [b1, b2, b3]|
+-----------------------------------------------------------------------+



In [70]:
spark.sql("SELECT variance(col) FROM VALUES (1), (2), (3) AS tab(col)").show()

+-------------+
|variance(col)|
+-------------+
|          1.0|
+-------------+



In [71]:
spark.sql("SELECT stddev_pop(col) FROM VALUES (1), (2), (3) AS tab(col)").show()

+-----------------+
|  stddev_pop(col)|
+-----------------+
|0.816496580927726|
+-----------------+



In [72]:
spark.sql("SELECT stddev_samp(col) FROM VALUES (1), (2), (3) AS tab(col)").show()

+----------------+
|stddev_samp(col)|
+----------------+
|             1.0|
+----------------+



# MLib

In [17]:
bInput = spark.read.format("parquet").load("/Users/aakash/Downloads/spark-code/data/binary-classification")\
  .selectExpr("features", "cast(label as double) as label")


bInput.show()

+--------------+-----+
|      features|label|
+--------------+-----+
|[3.0,10.1,3.0]|  1.0|
|[1.0,0.1,-1.0]|  0.0|
|[1.0,0.1,-1.0]|  0.0|
| [2.0,1.1,1.0]|  1.0|
| [2.0,1.1,1.0]|  1.0|
+--------------+-----+



In [ ]:
random forest
decision tree
knn
SVM
NN
logistic regression
bayes

In [18]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression()
lrModel = lr.fit(bInput)

print (lrModel.coefficients)
print (lrModel.intercept)

summary = lrModel.summary
print (summary.areaUnderROC)
summary.roc.show()
summary.pr.show()

[18.72238574166126,-0.5693688557340826,9.361192870830658]
-28.043295118689397
1.0
+---+------------------+
|FPR|               TPR|
+---+------------------+
|0.0|               0.0|
|0.0|0.3333333333333333|
|0.0|               1.0|
|1.0|               1.0|
|1.0|               1.0|
+---+------------------+

+------------------+---------+
|            recall|precision|
+------------------+---------+
|               0.0|      1.0|
|0.3333333333333333|      1.0|
|               1.0|      1.0|
|               1.0|      0.6|
+------------------+---------+



In [19]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression()
lr_model = lr.fit(bInput)
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))

Coefficients: [0.5625000000000061,-0.1250000000000011,0.28125000000000017]
Intercept: -0.26875000000000826
